<img src="/home/rsb6/Desktop/ComputerVision/eyes.png" alt="eyes points" width="1180">


In [25]:
# import the librarys
import cv2
import mediapipe as mp
import numpy as np
import time 


O EAR é dado pelo valor da distância euclidiana entre o ponto P2 e o ponto P6, mais o valor da distância euclidiana entre o ponto P3 e o ponto P5. Isso tudo sobre duas vezes a distância euclidiana entre o ponto P1 e o ponto P4.

$$EAR =\Large {\frac{(|| p_2 - p_6|| + ||p_3-p_5||)}{2*||p_1-p_2||}}$$


Aplicando a equação para as coordenadas de ambos os olhos

$$EAR_{right} =  \Large {\frac{||p_160-p_144|| + ||p_158-p_153||}{2*||p_33 - p_133||}}$$

$$EAR_{lefth} =  \Large {\frac{||p_385-p_380|| + ||p_387-p_373||}{2*||p_362 - p_263||}}$$ 

Algo recomendado para qualquer projeto é realizar uma análise: verificar possíveis problemas e pensar em ajustes para eles.

Vamos rodar a célula em que produzimos nosso código de verificação de EAR e fazer essa análise. Se eu fecho meus olhos, o tempo passa a ser contado e uma mensagem de alerta aparece caso permaneça com os olhos fechados por muito tempo.

No entanto, se eu estiver sorrindo, dando uma gargalhada, meus olhos ficarão semi cerrados. Isso faz com que o nosso código entenda que os olhos estão fechados. Vou fazer o teste sorrindo. A mensagem apareceu.

Precisamos verificar a abertura da boca, porque quase ninguém dorme sorrindo ou gargalhando. Geralmente, as pessoas dormem com o rosto relaxado. Agora fecharemos a janela de vídeo e calcularemos a abertura da boca. Para isso, utilizamos os pontos da boca.

<img src="/home/rsb6/Desktop/ComputerVision/mouth.png" alt="mouth points" width="1180">


Temos 8 pontos especificamente na boca. Nós realizaremos um cálculo similar ao EAR, utilizando esses pontos. O MAR ou Mouth Aspect Ratio (Em tradução livre: Proporção da boca) funciona exatamente como o EAR, mas para as coordenadas da boca.

Nosso primeiro passo é encontrar os pontos que estão indicados na imagem no MediaPipe. Então, temos que identificar esses pontos e mostrá-los na tela, assim como fizemos para o EAR.

Então, criaremos uma nova célula acima da célula de projeto. Dentro dela, faremos uma lista com os pontos da boca, exatamente os que são indicados na imagem do notebook: `82, 87, 13, 14, 312, 317, 308 e 78.`

O cálculo do índice de MAR é bem similar ao do índice de EAR. Isso porque a função MAR foi inspirada no EAR e descrita por Rishav Agarwal. Ambos os cálculos têm o mesmo objetivo: quantificar o nível de abertura.

MAR é uma sigla para Mouth Aspect Ratio e consegue medir o nível de abertura da boca através do cálculo das distâncias euclidianas. Os pontos a serem coletados são: `82, 87, 13, 14, 312, 317, 78 e 308` e se localizam no lábio superior, inferior e nas extremidades, como mostrado abaixo:

$$MAR = \Large \frac{||p_82-p_87||+||p_13-p_14||+||p_312-p_317||}{2*||p_78-p_308||}$$

### **Funtions Ultils**

In [16]:
def calculation_ear(face, p_eye_right, p_eye_left):
    try:
        face = np.array([[coord.x, coord.y] for coord in face]) 
        face_left  = face[p_eye_left,  :]
        face_right = face[p_eye_right, :]

        ear_left = (np.linalg.norm(face_left[0] - face_left[1]) + np.linalg.norm(face_left[2] - face_left[3]) 
                                                        / (2*(np.linalg.norm(face_left[4] - face_left[5]))))


        ear_right = (np.linalg.norm(face_right[0] - face_right[1]) + np.linalg.norm(face_right[2] - face_right[3]) 
                                                        / (2*(np.linalg.norm(face_right[4] - face_right[5]))))
        
    except:
        ear_left = 0
        ear_right = 0

    mean_ear = (ear_left + ear_right) / 2
    return mean_ear

In [22]:
# função que calcula as coordenadas de um membro (olhos e boca)
def coord_calculation(face, frame, p_coord, lenght, width):
    for id_coord, coord_xyz in enumerate(face):
        if id_coord in p_coord:
            coord_cv = mp_drawing._normalized_to_pixel_coordinates(coord_xyz.x,coord_xyz.y, lenght, width)
            cv2.circle(frame, coord_cv, 2, (255,0,0), -1)


In [26]:
def calculation_mar(face, p_mouth):
    try:
        face = np.array([[coord.x, coord.y] for coord in face])
        face_mouth = face[p_mouth, :]
        
        mar = (np.linalg.norm(face_mouth[0]-face_mouth[1]) + np.linalg.norm(face_mouth[2]-face_mouth[3]) + 
               + np.linalg.norm(face_mouth[4]-face_mouth[5])) / (2*(+ np.linalg.norm(face_mouth[6]-face_mouth[7]))) 
    except:
        mar = 0.0
    return mar

### **Executing code**

In [2]:
# coleta os pontos no rosto 
mp_drawing = mp.solutions.drawing_utils

# coleta a solução do facemesh
mp_face_mesh = mp.solutions.face_mesh


In [3]:
p_eye_left = [385, 380, 387, 373, 362, 263]
p_eye_right = [160, 144, 158, 153, 33, 133]


In [18]:
# Nessa célula, temos o p_boca, variável que guardará a lista, 
# igual aos valores dos pontos. Vamos rodar essa célula para que 
# a variável seja criada.
p_mouth = [82, 87, 13, 14, 312, 317, 78, 308]


In [4]:
# concatenar as coordenadas dos dois olhos
p_eyes = p_eye_left + p_eye_right
p_eyes

[385, 380, 387, 373, 362, 263, 160, 144, 158, 153, 33, 133]

In [28]:
ear_limiar = 0.18

sleep = 0

cam_option = -1

cap = cv2.VideoCapture(cam_option)

# realiza a chamada do método para a detecção da face
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as facemesh:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("No cam frame\n")
            continue
        
                 
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        output_facemesh = facemesh.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)


        width, lenght, _ = frame.shape
        
        try:    
            for face_landmarks in output_facemesh.multi_face_landmarks:
                # coleta os pontos correspondentes as coordenadas do rosto
                mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec = mp_drawing.DrawingSpec(color=(255,102,102),thickness=1,circle_radius=1),
                    connection_drawing_spec = mp_drawing.DrawingSpec(color=(102,204,0),thickness=1,circle_radius=1)),

            face = face_landmarks.landmark
            coord_calculation(face, frame, p_eyes, lenght, width)
            coord_calculation(face, frame, p_mouth, lenght, width)

       
            ear = calculation_ear(face, p_eye_right, p_eye_left)
            
            #Desenha um retangulo preenchido
            cv2.rectangle(frame, (0,1), (170, 90), (58,58,55), -1)
            
            # Adicionando texto
            cv2.putText(frame, f"EAR: {round(ear, 2)}", (1, 24),
                        cv2.FONT_HERSHEY_DUPLEX,
                        0.7, (255, 255, 255), 2)

            mar = calculation_mar(face, p_mouth)
            # Adicionando texto
            cv2.putText(frame, f"MAR: {round(mar, 2)}", (1, 50),
                        cv2.FONT_HERSHEY_DUPLEX,
                        0.7, (255, 255, 255), 2)
            
            if ear < ear_limiar:
                t_init = time.time() if sleep == 0 else t_init
                sleep = 1
                
            if sleep == 1 and ear >= ear_limiar:
                sleep = 0
            t_final = time.time()
            
            _time = (t_final-t_init) if sleep == 1 else 0.0        
            cv2.putText(frame, f"Time: {round(_time, 3)}", (1, 80),
                                    cv2.FONT_HERSHEY_DUPLEX,
                                    0.7, (255, 255, 255), 2)
            
            if _time >= 1.5:
                cv2.rectangle(frame, (30, 400), (610, 452), (109, 233, 219), -1)
                cv2.putText(frame, f'So many time with close eye!', (80, 435),
                cv2.FONT_HERSHEY_DUPLEX,
                0.85, (58, 58, 55), 1)

                
        except:             
            pass
        
        cv2.imshow('CAM', frame)
        if cv2.waitKey(10) & 0xFF == ord('c'):
            break
cap.release()
cv2.destroyAllWindows()
    